# Convolutional Neural Network with Fractional Max Pooling#
## Full Process Jupyter Notebook ##

### Introduction ###
** This model is inspired by the all-convolutional network

In [1]:
import tensorflow as tf
import numpy as np
import pickle
import random as rd

In [2]:
def unpickle(file):
   
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
x_train=np.array([])
y_train=np.array([])
x_val=np.array([])
y_val=np.array([])
for i in range(1,5):
    
    batch = unpickle("/data/cifar-10-batches-py/data_batch_%d"%(i))
    if len(x_train)==0 & len(y_train)==0:
        x_train = batch[b'data']
        y_train = batch[b'labels']
    else:
        x_train = np.concatenate((x_train, batch[b'data'])) 
        y_train = np.concatenate((y_train, batch[b'labels']))

v_batch = unpickle("/data/cifar-10-batches-py/data_batch_5")
x_val = v_batch[b'data']
y_val = v_batch[b'labels']

In [4]:
y_train0 = np.empty((0,10), int)
for y in y_train:
    indi= [1 if i==y else 0 for i in range(0,10)]
    y_train0 = np.append(y_train0, np.array([indi]), axis=0)
y_val0 = np.empty((0,10), int)
for y in y_val:
    indi= [1 if i==y else 0 for i in range(0,10)]
    y_val0 = np.append(y_val0, np.array([indi]), axis=0)

In [5]:
test_batch = unpickle("/data/cifar-10-batches-py/test_batch")
x_test = test_batch[b'data']
y_test = test_batch[b'labels']

In [6]:
y_test0 = np.empty((0,10), int)
for y in y_test:
    indi= [1 if i==y else 0 for i in range(0,10)]
    y_test0 = np.append(y_test0, np.array([indi]), axis=0)

## Model

In [7]:
def compute_logits(x):
    """CNN-FMP network model"""
    
    #input x.shape = [,32*32*3]
    x_image = tf.reshape(x,[-1,32,32,3])
    
    #define some constants, stack size for convolutional layers
    n1=320
    n2=640
    n3=960
    n4=1280
    n5=1600
    n6=1920

    
    #Block1
    #cnn_1
    W_conv1 = tf.get_variable('W_conv1', shape=[2, 2, 3, n1])
    b_conv1 = tf.get_variable('b_conv1', shape=[n1])
    h_conv1 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME'), b_conv1))
    #cnn_2
    W_conv2 = tf.get_variable('W_conv2', shape=[2, 2, n1, n1])
    b_conv2 = tf.get_variable('b_conv2', shape=[n1])
    h_conv2 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(h_conv1, W_conv2, strides=[1, 1, 1, 1], padding='SAME'), b_conv2))
    #fractional_max_pooling_1 (pooling size is sqrt(2)=1.414)
    fmp_1=tf.nn.fractional_max_pool(h_conv2,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]
    

    
    #block_2
    #dropout rate = 0.1 
    #cnn_3
    W_conv3 = tf.get_variable('W_conv3', shape=[2, 2, n1, n2])
    b_conv3 = tf.get_variable('b_conv3', shape=[n2])
    h_conv3 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_1, W_conv3, strides=[1, 1, 1, 1], padding='SAME'), b_conv3))
    #dropout_1
    d_1=tf.nn.dropout(h_conv3,0.9)
    #cnn_4
    W_conv4 = tf.get_variable('W_conv4', shape=[2, 2, n2, n2])
    b_conv4 = tf.get_variable('b_conv4', shape=[n2])
    h_conv4 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_1, W_conv4, strides=[1, 1, 1, 1], padding='SAME'), b_conv4))
    #dropout_2
    d_2=tf.nn.dropout(h_conv4,0.9)
    #fractional_max_pooling_2
    fmp_2 = tf.nn.fractional_max_pool(d_2,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]
    
    
    
    #block_3
    #dropout=0.2
    #cnn_5
    W_conv5 = tf.get_variable('W_conv5', shape=[2, 2, n2, n3])
    b_conv5 = tf.get_variable('b_conv5', shape=[n3])
    h_conv5 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_2, W_conv5, strides=[1, 1, 1, 1], padding='SAME'), b_conv5))
    #dropout_3
    d_3=tf.nn.dropout(h_conv5,0.8)
    #cnn_6
    W_conv6 = tf.get_variable('W_conv6', shape=[2, 2, n3, n3])
    b_conv6 = tf.get_variable('b_conv6', shape=[n3])
    h_conv6 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_3, W_conv6, strides=[1, 1, 1, 1], padding='SAME'), b_conv6))
    #dropout_4
    d_4=tf.nn.dropout(h_conv6,0.8)
    #fractional_max_pooling_3
    fmp_3 = tf.nn.fractional_max_pool(d_4,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]
    

    
    #block_4
    #dropout=0.3
    #cnn_7
    W_conv7 = tf.get_variable('W_conv7', shape=[2, 2, n3, n4])
    b_conv7 = tf.get_variable('b_conv7', shape=[n4])
    h_conv7 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_3, W_conv7, strides=[1, 1, 1, 1], padding='SAME'), b_conv7))
    #dropout_5
    d_5=tf.nn.dropout(h_conv7,0.7)
    #cnn_8
    W_conv8 = tf.get_variable('W_conv8', shape=[2, 2, n4, n4])
    b_conv8 = tf.get_variable('b_conv8', shape=[n4])
    h_conv8 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_5, W_conv8, strides=[1, 1, 1, 1], padding='SAME'), b_conv8))
    #dropout_6
    d_6=tf.nn.dropout(h_conv8,0.7)
    #fractional_max_pooling_4
    fmp_4 = tf.nn.fractional_max_pool(d_6,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]
    
    
    #block_5
    #dropout=0.4
    #cnn_9
    W_conv9 = tf.get_variable('W_conv9', shape=[2, 2, n4, n5])
    b_conv9 = tf.get_variable('b_conv9', shape=[n5])
    h_conv9 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_4, W_conv9, strides=[1, 1, 1, 1], padding='SAME'), b_conv9))
    #dropout_7
    d_7=tf.nn.dropout(h_conv9,0.6)
    #cnn_10
    W_conv10 = tf.get_variable('W_conv10', shape=[2, 2, n5, n5])
    b_conv10 = tf.get_variable('b_conv10', shape=[n5])
    h_conv10 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_7, W_conv10, strides=[1, 1, 1, 1], padding='SAME'), b_conv10))
    #dropout_8
    d_8=tf.nn.dropout(h_conv10,0.6)
    #fractional_max_pooling_5
    fmp_5 = tf.nn.fractional_max_pool(d_8,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]
   


    #block_6
    #dropout=0.5
    #cnn_11
    W_conv11 = tf.get_variable('W_conv11', shape=[2, 2, n5, n6])
    b_conv11 = tf.get_variable('b_conv11', shape=[n6])
    h_conv11 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_5, W_conv11, strides=[1, 1, 1, 1], padding='SAME'), b_conv11))
    #dropout_9
    d_9=tf.nn.dropout(h_conv11,0.5)
    #cnn_12
    W_conv12 = tf.get_variable('W_conv12', shape=[2, 2, n6, n6])
    b_conv12 = tf.get_variable('b_conv12', shape=[n6])
    h_conv12 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_9, W_conv12, strides=[1, 1, 1, 1], padding='SAME'), b_conv12))
    #dropout_10
    d_10=tf.nn.dropout(h_conv12,0.5)
    #fractional_max_pooling_6
    fmp_6 = tf.nn.fractional_max_pool(d_10,pooling_ratio=[1.0, 1.414, 1.414, 1.0])[0]
    
    
    
    #lcnn with stride=2 
    W_conv = tf.get_variable('W_conv', shape=[2, 2, n6, n6])
    b_conv = tf.get_variable('b_conv', shape=[n6])
    h_conv = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(fmp_6, W_conv, strides=[1, 2, 2, 1], padding='SAME'), b_conv))

    # fc layer to logits 10
    h_flat = tf.reshape(h_conv, [-1, 1*1*n6])
    W_fc1 = tf.get_variable('W_fc1', shape=[1*1*n6, 10])
    b_fc1 = tf.get_variable('b_fc1', shape=[10])
    #layer of output
    logits = tf.add(tf.matmul(h_flat, W_fc1), b_fc1, name='logits')
    
    return(logits)

def compute_cross_entropy(logits, y):
    """compute the prediction and cross_entropy of model"""
    # This function is used from the in-class example code
    numerical_instability_example = 0
    if numerical_instability_example:
        y_pred = tf.nn.softmax(logits, name='y_pred') 
        cross_ent = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices=[1]))
    else:
        sm_ce = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits, name='cross_ent_terms')
        cross_ent = tf.reduce_mean(sm_ce, name='cross_ent')
    return cross_ent

def compute_accuracy(logits, y):
    "compare prediction to labels"
    prediction = tf.argmax(logits, 1, name='pred_class')
    true_label = tf.argmax(y, 1, name='true_class')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, true_label), tf.float32))
    return accuracy
    
    

In [ ]:
dir_name = 'logs/scratch04x/summary'
with tf.Graph().as_default():

    x = tf.placeholder(tf.float32, [None, 32*32*3], name='x')
    y = tf.placeholder(tf.float32, [None, 10], name='y')

    
 
    with tf.name_scope('model'):
        logits = compute_logits(x)
    with tf.name_scope('loss'):
        loss = compute_cross_entropy(logits=logits, y=y)
    with tf.name_scope('accuracy'):
        accuracy = compute_accuracy(logits, y)
    
    with tf.name_scope('opt'):

        opt = tf.train.AdamOptimizer(1e-4)
        train_step = opt.minimize(loss)
    
    with tf.name_scope('summaries'):
        # create summary for loss and accuracy
        tf.summary.scalar('loss', loss) 
        tf.summary.scalar('accuracy', accuracy)
        # create summary for logits
        tf.summary.histogram('logits', logits)
        # create summary for input image
        tf.summary.image('input', tf.reshape(x, [-1, 32, 32, 3]))
    
        summary_op = tf.summary.merge_all()
    
    with tf.Session() as sess:
        summary_writer = tf.summary.FileWriter(dir_name, sess.graph)
        summary_writer_train = tf.summary.FileWriter(dir_name+'/train', sess.graph)
        summary_writer_test = tf.summary.FileWriter(dir_name+'/test')
        summary_writer_val = tf.summary.FileWriter(dir_name+'/val')
        sess.run(tf.global_variables_initializer())
        batch=0
        for i in range(10001):
            
             
            _ , summary = sess.run((train_step, summary_op),
                                feed_dict={x: x_train[100*batch:100*(batch+1)], y: y_train0[100*batch:100*(batch+1)]})
            batch=batch+1
            if batch == 50:
                batch=0
            if i%10==0:
                t=rd.sample(range(0,10000),1000)
                summary_writer_train.add_summary(summary, i)
                (val_error, summary_t) = sess.run((accuracy,summary_op), {x:x_val[t], y:y_val0[t]})
                summary_writer_val.add_summary(summary_t, i)
                if i%1000 == 0:
                    print("\rAfter step {0:3d}, valiation accuracy {1:0.4f}".format(i, val_error), flush=True)
        #calculate test error
        test_error=0
        for i in range(0,10):
            test_error = test_error+sess.run(accuracy,{x:x_test[1000*i:1000*(i+1)],y:y_test0[1000*1:1000*(i+1)]})
        print("\rFinal test accuracy is{1:0.4f}".format(test_error))

    

After step   0, test accuracy 0.0970
After step 1000, test accuracy 0.2870
After step 2000, test accuracy 0.4960
After step 3000, test accuracy 0.5020
After step 4000, test accuracy 0.5730
After step 5000, test accuracy 0.5500
After step 6000, test accuracy 0.5760
After step 7000, test accuracy 0.6140
After step 8000, test accuracy 0.6080
After step 9000, test accuracy 0.6100
After step 10000, test accuracy 0.5910
After step 11000, test accuracy 0.5580
After step 12000, test accuracy 0.5910
After step 13000, test accuracy 0.5660
